In [20]:
# libraries
import os
import requests
import threading
import pandas as pd
from bs4 import BeautifulSoup as bs4
from requests.adapters import HTTPAdapter

# from Project
from GetSheetStatement import Constractv3


class MOPS(object):

    def __init__(self, sheet: str, sid: str, year: int, season: int):
        #super().__init__()
        self.sheet = Constractv3.FINANCIAL_STATMENT[sheet]
        self.url = Constractv3.MOPS_URL.format(sheet=self.sheet)
        Constractv4.HEADERS['Referer'].format(sheet=self.sheet)
        self.header = Constractv3.HEADERS
        self.payload = Constractv3.PAYLOAD
        self.payload ["co_id"] = sid
        self.payload["year"] = int(year)
        self.payload["season"] = f"0{season}"
        self.file_path = Constractv3.FILE_PATH.format(
            sheet=sheet, sid=self.payload["co_id"],year=self.payload["year"] + 1911, season=self.payload["season"])
        if not os.path.exists(sheet):
            os.mkdir(sheet)

        print(f"INIT Thread: {sheet}\nURL: {self.url}\nFILE_PATH: {self.file_path}\n")
        pass

    def run(self):
        # set session retry option
        web_ss = requests.session()
        ss_adapter = HTTPAdapter(max_retries=3)
        web_ss.mount("https://", adapter=ss_adapter)
        try:
            # start get html
          
            res = web_ss.post(url=self.url, headers=self.header, data=self.payload, timeout=3)
            
            # check iff: Response OK
            if res.status_code == 200:
                soup = bs4(res.text, "html.parser")
                tables = soup.select('table.hasBorder')
                #print(tables)
                return tables
            
        except Exception as e:
            print(f"Get HTML FAIL: {e}", end="\n\n")


#B is 資產負債表
#sheet = 'B'

#P is 損益表
#sheet= 'P' 

#C is 現金流量表
#sheet= 'C'

#S is 股東權益表
sheet= 'S' 


sid = '1101'
year = 108
season= 4

MOPS=MOPS(sheet, sid, year, season)
result=MOPS.run()

#print(result[0])


even = result[0].select('td.even')
odd = result[0].select('td.odd')


#for i, v in enumerate(even):
#    print(i, v.text)
    
for i, v in enumerate(odd):
    print(i, v.text)


# %k
# B: 1, 2, 3 = 5
# B: 4 = 7
# P: 1, 2, 3, 4 = 5
# C = 3

if sheet=='C':
    k = 3
else:
    if season==4 or sheet == 'P':
        k=5
    else:
        k=7
        

INIT Thread: S
URL: https://mops.twse.com.tw/mops/web/ajax_t164sb06
FILE_PATH: ./S/1101-2019-04.json

0 普通股股本
1 特別股股本
2 債券換股權利證書
3 預收股本
4 待分配股票股利
5 其他
6 股本合計
7 資本公積
8 法定盈餘公積
9 特別盈餘公積
10 未分配盈餘（或待彌補虧損）
11 保留盈餘合計
12 國外營運機構財務報表換算之兌換差額
13 透過其他綜合損益按公允價值衡量之金融資產未實現評價(損)益
14 備供出售金融資產未實現利益（損失）
15 現金流量避險中屬有效避險部分之避險工具利益（損失）
16 國外營運機構淨投資避險中屬有效避險部分之避險工具利益（損失）
17 指定按公允價值衡量之金融負債信用風險變動影響數
18 確定福利計畫再衡量數
19 避險工具之利益（損失）
20 不動產重估增值
21 與待出售非流動資產(或處分群組)直接相關之權益
22 與待分配予業主之非流動資產(或處分群組)直接相關之權益
23 採用覆蓋法重分類之其他綜合損益
24 其他
25 其他權益項目合計
26 庫藏股票
27 歸屬於母公司業主之權益總計
28 非控制權益
29 權益總額
30 　追溯適用及追溯重編之影響數
31                       0
32                       0
33                       0
34                       0
35                       0
36                       0
37                       0
38                       0
39                       0
40                       0
41                       0
42                       0
43                       0
44                       0
45                       0
46                       

In [ ]:
columns_odd=[]

for i, v in enumerate(even):
    if i% k ==0 :
        columns_odd.append(v.text.strip())
#print(columns_odd , len(columns_odd))

columns_even=[]
for i, v in enumerate(odd):
    if i% k ==0 :
        columns_even.append(v.text.strip())
        
#print(columns_even , len(columns_even))

columns=[]
for _o, _e in zip(columns_odd, columns_even):
    columns.append(_o)
    columns.append(_e)
    
print(columns, len(columns))

In [ ]:
content_odd=[]
for i, v in enumerate(even):
    if i% k ==0 :
        content_odd.append(even[i+1].text.strip().replace(',',''))   
#print(content_odd)

content_even=[]
for i, v in enumerate(odd):
    if i% k ==0 :
        content_even.append(odd[i+1].text.strip().replace(',',''))   
#print(content_even)

content=[]
for _o, _e in zip(content_odd, content_even):
    content.append(_o)
    content.append(_e)
    
#print(content, len(content))

Newcolumns=['sID','Quarter']
Newcontent=[sid, season]
Newcolumns.extend(columns)
Newcontent.extend(content)


df=pd.DataFrame(columns=Newcolumns)

df.loc[1]=Newcontent


#json = df.to_json(orient='records')

df.to_json(BSS.file_path, orient='records')

df